In [0]:
# %load_ext autoreload
# %autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0


# Tables

---

## mysql sources
- customers
- items

## bronze tables
- customers_bronze
- items_bronze

In [0]:
%run "../../config/setup"


### Python Utils

Python util functions for ingesting mysql and writing to delta

In [0]:
from notebooks.utils.mysql_ingestion import *

In [0]:
from config.settings import *
from pyspark.sql.functions import input_file_name, current_timestamp

dbutils.widgets.text("database", RDS_DATABASE)
dbutils.widgets.text("host", RDS_HOST)
dbutils.widgets.text("user", RDS_USER)
dbutils.widgets.text("port", RDS_PORT)
dbutils.widgets.text("password", RDS_PASSWORD)


### SQL code for ingesting customers data from RDS

In [0]:
%sql

CREATE OR REPLACE TEMP VIEW customers_tv
USING mysql
OPTIONS (
  dbtable "customers",
  user "${user}",
  password "${password}",
  host "${host}",
  port "${port}",
  database "${database}",
  driver "org.mariadb.jdbc.Driver"
);

CREATE OR REPLACE TABLE customers_bronze AS
SELECT *, current_timestamp() AS ingest_ts FROM customers_tv;

-- SELECT * FROM customers_bronze;


### Python code for ingesting customers data from RDS

In [0]:

# def mysql_to_dataframe(spark, table_name):
#     return ingest_mysql_table(
#         spark,
#         table_name,
#         {
#             "host": RDS_HOST,
#             "port": "3306",
#             "database": RDS_DATABASE,
#             "user": RDS_USER,
#             "password": RDS_PASSWORD,
#         }
#     )

In [0]:
# %python

# customers = mysql_to_dataframe(spark, "customers")

# write_ingested_mysql_dataframe(customers, "customers_bronze")

# display(spark.table('customers_bronze'))

In [0]:
%sql
-- grant select on customers_bronze to devs
GRANT SELECT ON TABLE customers_bronze TO devs;


### SQL code for ingesting items data from RDS

In [0]:
%sql

CREATE OR REPLACE TEMP VIEW items_tv
USING mysql
OPTIONS (
  dbtable "items",
  user "${user}",
  password "${password}",
  host "${host}",
  port "${port}",
  database "${database}",
  driver "org.mariadb.jdbc.Driver"
);

CREATE OR REPLACE TABLE items_bronze AS
SELECT *, current_timestamp() AS ingest_ts FROM items_tv;

SELECT * FROM items_bronze;


### Python code for ingesting items data from RDS

In [0]:
# %python

# items = mysql_to_dataframe(spark, "items")

# write_ingested_mysql_dataframe(items, "items_bronze")

# display(spark.table('items_bronze'))

In [0]:
%sql
-- grant select on items_bronze to devs
GRANT SELECT ON TABLE items_bronze TO devs;